In [16]:
url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"

In [14]:
import pandas as pd
import os
from tqdm.auto import tqdm
import requests
import pandas as pd
import json
import psycopg2

In [2]:
v2_dict = pd.read_csv("./reverse_connected_dicts/to_release/merge_reverse_en_ru_fr_vs_ru_fr_en.csv")
words_to_be_handled = list(set(v2_dict['fr']))

big_token_dict = {
 "vertolet_token" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
   "bbk_token" : "dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc", 
"vasya_2019" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
"vasya_2011" : "dict.1.1.20191105T083743Z.8b9cdebb9755fa57.e42afa31c184e2ced2305b5452809e17bc19c920",
"fanyi0" : "dict.1.1.20191105T095108Z.82f72da803eeedc2.ada6293a0591bed0222d3a8045bd722915f025bb",
"fanyi00" : "dict.1.1.20191105T103102Z.3e0f76aef01d786a.92fea3781473b7c432aa49ee1f3879cb95181a63",
"prvd0" : "dict.1.1.20191105T103348Z.960e0892b02b628b.5746d7d95af92a16d15941b24410707bbb7568c4",
"prvd00" : "dict.1.1.20191105T105231Z.520f147163f275cd.f329913de7b0a719762e89ef84914cbe8c3f3d25",
"prvdfin0": "dict.1.1.20191105T105704Z.a6f94da915862b61.76003fa911b16bbad05aa655d5be9ea1f78b3e1c" 
}

words_to_be_handled[:10]


["c'est sûr",
 'immédiate',
 "s'allier",
 'temps de pose',
 'hémérocalle',
 'hybridité',
 'illicéité',
 'trente-six',
 'dans les parages',
 'antillais']

In [28]:
def get_handled_words(directory_with_handled_files):
    handled_words_from_folder = []
    for word in os.listdir(directory_with_handled_files):
        #print(word.split('.')[0])
        handled_words_from_folder.append(word.split('.')[0])
    handled_words_from_folder = set(handled_words_from_folder)
    #print(check_word, "word_in", check_word in handled_words_from_folder)
    #print("akready_handled_from_directory", len(handled_words_from_folder))
    return handled_words_from_folder

a = get_handled_words("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2")
a

{'exciter',
 'somalien',
 'mes',
 'le grec',
 'francophonie',
 'barista',
 'créant',
 'trop-plein',
 'déranger',
 'capri',
 'grande bretagne',
 'No',
 'ouganda',
 'solde',
 'Mer',
 'Elise',
 'dist',
 'Syrie',
 'Héron',
 'lequel',
 'assez de',
 'gentil',
 'se gonfler',
 'polluant',
 'ardennes',
 "d'une branche d'industrie",
 'Krishna',
 'infliction',
 'opalin',
 'rectorat',
 'éthiopie',
 'Cincinnati',
 'imagerie',
 'beach',
 'étonnement',
 'géranium',
 'non volatile',
 'lieber',
 'Incas',
 'juliette',
 'flirter',
 'hexagonal',
 'sabine',
 "à l'arrière de",
 'pluralisme',
 'champ du regard',
 'disparaître',
 'messie',
 'secrétaire particulier',
 'revenir sur',
 'Nairobi',
 'swapping',
 'vipère',
 'Santiago',
 'Milan',
 'stewart',
 'britt',
 'incarne',
 'western',
 'deuxième langue',
 "se lier d'amitié",
 'interdise',
 'empaqueter',
 'Malais',
 'av',
 'incendie de forêt',
 'cap',
 'Tage',
 'appendice',
 'miniaturiste',
 'vestiges',
 'rapports sexuels',
 'urbaniser',
 'sambo',
 'petits gar

In [25]:
def get_y_lookup_json(from_lang_words_list, from_lang, to_lang, token_dict, directory_with_handled_files):
    tokens_names_list = []
    tokens_list = []
    for token_name, token in token_dict.items():
        tokens_names_list.append(token_name)
        tokens_list.append(token)
    token_index = 0 
    not_handled_words_list = []
    
    save_folder = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2"
   
    handled_words_from_folder = get_handled_words(directory_with_handled_files)
    handled_words_from_folder = set(handled_words_from_folder)
    handled_words_from_folder_2 = get_handled_words(save_folder)
    handled_words_from_folder_2 = set(handled_words_from_folder_2)
    handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
    print("handled_all_words", len(handled_words_from_folder))

    count = 0 
    print("going_to_use token", tokens_names_list[token_index])
    skipped_words_count = 0
    handled_words_count = 0
    for fr_word in tqdm(from_lang_words_list):
        #print(fr_word)
        isnan = False
        #print(count, start_from_count)
        try:
            isnan = math.isnan(fr_word)
        except:
            pass
        if fr_word != "no_equality" and isnan == False and fr_word not in handled_words_from_folder:
            lang_pair = from_lang + "-" + to_lang
            headers = {"key":tokens_list[token_index],
                  "lang":lang_pair,"text":fr_word}
            ddd = requests.get(url, headers).json()
            if 'message' in ddd and ddd['message'] == "Limit of daily requests exceeded":
                token_index += 1
                print("LIMIT REACHED, switch to token",tokens_names_list[token_index] )
                not_handled_words_list.append(fr_word)
            try:
                save_loc = os.path.join(save_folder, fr_word + ".json")
            except:
                print(fr_word, "is ambigious float or smth")
                continue
            try:
                with open(save_loc, 'w') as f:
                    json.dump(ddd, f, indent = 4, ensure_ascii=False)
                    #print("saved at ", save_loc)
            except Exception as E:
                print(fr_word, E)
            #time.sleep(0.0001)
            handled_words_count += 1
        else:
            skipped_words_count += 1
            #print(fr_word, "===>>>>skipped")
            pass
            
        count += 1
        if count %500 == 0:
            handled_words_from_folder = get_handled_words(directory_with_handled_files)
            handled_words_from_folder = set(handled_words_from_folder)
            handled_words_from_folder_2 = get_handled_words(save_folder)
            handled_words_from_folder_2 = set(handled_words_from_folder_2)
            handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
            print("handled_words_from_folder", len(handled_words_from_folder))
            print(handled_words_count, "words handled", skipped_words_count, "words skipped\n",)     
    return not_handled_words_list
        

In [4]:
small_token = {

"prvd0" : "dict.1.1.20191105T103348Z.960e0892b02b628b.5746d7d95af92a16d15941b24410707bbb7568c4",
"prvd00" : "dict.1.1.20191105T105231Z.520f147163f275cd.f329913de7b0a719762e89ef84914cbe8c3f3d25",
"prvdfin0": "dict.1.1.20191105T105704Z.a6f94da915862b61.76003fa911b16bbad05aa655d5be9ea1f78b3e1c" 
}

In [19]:
not_handled_words

['inconditionné', 'âme-sœur']

In [29]:
handled = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en"
#from_ind = 1480+382+133+1858+15
#to_ind = 25000
from_ind = 25000
to_ind = len(words_to_be_handled)
not_handled_words = get_y_lookup_json(words_to_be_handled[from_ind:to_ind], "fr", "en", small_token,
                  directory_with_handled_files = handled)



handled_all_words 47103
going_to_use token prvd0


handled_words_from_folder 47103
0 words handled 500 words skipped

LIMIT REACHED, switch to token prvd00
LIMIT REACHED, switch to token prvdfin0
handled_words_from_folder 47107
3 words handled 997 words skipped

handled_words_from_folder 47109
4 words handled 1496 words skipped

handled_words_from_folder 47114
7 words handled 1993 words skipped

handled_words_from_folder 47122
11 words handled 2489 words skipped

handled_words_from_folder 47127
15 words handled 2985 words skipped

handled_words_from_folder 47135
21 words handled 3479 words skipped

handled_words_from_folder 47148
28 words handled 3972 words skipped

handled_words_from_folder 47160
35 words handled 4465 words skipped

handled_words_from_folder 47173
43 words handled 4957 words skipped

24 h/24 [Errno 2] No such file or directory: '/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2/24 h/24.json'
handled_words_from_folder 47185
50 words handled 5450 words skipped

handled_words_from_folder 47202
58 words hand

In [20]:
handled = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en"
#from_ind = 25000 + 22692 
#to_ind = len(words_to_be_handled) - 8000
not_handled_words = get_y_lookup_json(not_handled_words[:], "fr", "en", small_token,
                  directory_with_handled_files = handled)

